# 1. Uvrščanje glasbe v žanr
## 1.1 Obdelava podatkov

Pesmi bomo razvrscali v 5 najbolj popularnih zanrov na platformi spotify.

Najbolj popularne zanre najdemo na spletni strani Every Noise: https://everynoise.com/everynoise1d.cgi?scope=all

Vzeli bomo rap, pop, rock, latin, edm in se dodatno klasiko - skupaj 6 zanrov.

Podatke bomo dobil s platforme spotify developer, s pomocjo knjiznjice spotipy v pythonu. 

Pridobili bomo po 15000-20000 pesmi iz vsakega od 6 zanrov, ki smo jih dolocili.

Predvidevam, da naj bi v osnovi naj bi zanre locili na:
- Classical - instrumentalno in akusticnost (atribut Instrumentalness, acousticness)
- Rap - veliko govora (atribut Speechiness)
- Rock - nemoremo plesati (atribut danceability)
- EDM - high temp, high energy (atribut tempo, energy)
- Latin - mozno plesati (atribut danceability)
- pop - vse kar ostane

S pomocjo knjiznjice spotipy in metode search bomo iskali pesmi/albume/playliste/artiste za vsak zanr in jih dodali v data set v 1 od glavnih 6 zanrov. To bomo naredili v python skripti in podatke shranili v .csv datoteko.

Uporaba GenerateData.py skripte. 

1. Pridobil iz spotifya po 200 artistov iz vsakega zanra (skupaj 1200)
2. Od vsakega artista pridobil po 10 albumov (skupaj 12000)
3. Iz vsakega albuma dobil 10 pesmi (skupaj 120000)
4. Izlocil vse duplikate

Imamo 83698 pesmi. 

<img src="pictures/Genre/num_tracks.png" width="400"/>

<img src="pictures/Genre/duration_outliers.png" width="800"/>
<img src="pictures/Genre/outliers_minMax.png" width="500"/>
<img src="pictures/Genre/outliersRemoved_minMax.png" width="500"/>

<img src="pictures/Genre/audioF_14.png" width="900"/>
<img src="pictures/Genre/audioF_58.png" width="900"/>
<img src="pictures/Genre/audioF_912.png" width="900"/>

<img src="pictures/Genre/audioF_boxplot_16.png" width="900"/>

Se enkrat povzamemo, pri rapu je nekoliko vec govora kot pri drugih zanrih. Pri moznosti plesanja v pesmi, se od drugih zanrov locita najbolj klasika in rock. Energija je najmanjsa v pesmih klasike, najvecja pa pri rocku in EDM. Insturmentalnost in akusticnost sta bolj znacilna za klasiko, drugi zanri imajo delez teh 2 atributov blizu 0. Rock pesmi imajo vec moznosti, da so izvedene v zivo pred publiko.

<img src="pictures/Genre/audioF_boxplot_712.png" width="900"/>

Pocutje v klasikah je bolj zalostno, bolj veselo pa je v pesmih latino zanra, to opazimo pri atributu pocutja. Pri glasnosti najbloj izstopa klasika, ki je tisja od drugih zanrov. Pri casu trajnja iztopa klasika, ki ima vecji razopn trajanja, vsi drugi zanri pa imajo manjsi razpon. Tempo je najbloj specificen pri EDM zanru, vsi drugi zanri imajo vecji razpon, klasika pa pa se od drugih loci po nizjem tempu. S tonaliteto (mode) in kljucem (key) pesmi pa nimamo nekega vzorca, po katerem bi lahko razlikovali zanre. Ta 2 atributa nista pomepomembna pri kalsifikaciji in ju bomo odstranili.

<img src="pictures/Genre/audio_features_correlation.png" width="900"/>

Pri grafu korelacij atributov, opazimo, da sta atributa energy in loudness zelo povezana. Vecja kot je glasnost pesmi, vecja bo energija.

## 1.2 Klasifikacija zanrov

V prejsnjem koraku (Genre.pyb) smo v originalnem data setu (83000 pesmi) pregledali in poiskali vzorce v vseh atributih.
Po analizi, smo obdrzali le smiselne, tisti, ki so najbolj razlicni od drugih.

Iz vizualizacij in vzorceh je najbolj izstopala klasika, zato predvidevamo, da jo bomo najlazje locili od drugih. (Bo kasifikacijska tocnost tega zanra vecja od drugih). Opazili smo tudi, da se EDM in rap po dolocenih atributih kar dobro locita od drugih zanrov. Najmanj vzorcev/specificnosti smo opazili pri zanru pop. V nobenem atributu ni izstopal, zato predvidevam, da bo pop tezje loziti od drugih zanrov. 

Za klasifikacijo bomo uporabili vec klasifikatorjev: 
1. Klasifikacijsko drevo 
2. KNN (n-najbljznjih sosedov)
3. Nakljucni gozdovi
4. Gradient boosting klasifikator

Da bi klasifikacijsko model dobro napovedoval, more imeti klasifikacijsko tocnost vec kot nakljucno napovedovanje.

Nakjucno napovedovanje: 6 zanrov / 1 = 0.17 = 17% tocnost. Model mora imeti kalsifikacijsko tocnost vec koy 17% da bo uporaben.

<img src="pictures/Genre/matrix_4.png" width="900"/>
<img src="pictures/Genre/classifiersAccuracy.png" width="500"/>

REZULTATI:

Kot smo predvidevali, klasifikatorji najnatancnejse uvrscajo pesmi klasike, ker je seveda najbolj drugacna od drugih zanrov. 
Klasifikacijska tocnost klasike je 93%. Vidimo, da je 2. najbolj tocno napovedan zanr rock s tocnostjo 70%, opazimo tudi, da je v rock zanr napacno napovedanih najvec pesmi iz zanra pop in latin. Po natancnosti za rockom sledi rap s 67% natancnostjo, isto kot pri rocku vidimo tudi pri rapu, da je najvec napacno napovedanih pesmi iz zanra pop in latin.

Sledita EDM z natancnostjo 60% in latino z natancnostjo 57%. Opazimo, da imam pop najmanjso natanconst 33%. Ze iz vizualizacije smo ugotovilo, da bomo pop locili od drugih zanrov tezko, ker nikjer ne izstopa (se razlikuje od drugih zanrov). Zdaj lahko potrdimo, da je pop najtezje napovedati od vseh zanrov, ker ima lastnosti vseh drugih zanrov. 

KLASIFIKATORJI:
  
Primerjamo se klasifikatorje med seboj. Opazimo, da KNN klasifikacija za 3% bolje napoveduje kot odlocitveno drevo. Najvecjo izboljsavo opazimo pri napovedovanju latino pesmi. Se vecjo razliko o tocnosti napovedovanja opazimo pri Nakljucnih gozdovih. V primerjavi s KNN nakljucni gozdovi veliko bolje napovedujejo pesmi rocka in rapa.

Gradient boosting kalsifikator pa je po natancnosti blizu klasifikatorju nakljucnih gozdovih, amapak je za izvedbo veliko bolj casovno zahteven.

# 2. Song recommendation

## 2.1 Primerjava playlistov (uporabnikov) in pesmi z vekotrjem atributov

1. Izracun vekotorja - povprecje (mean) ALI sestevek (sum) vseh vrstic 
2. Primerjava 2 vektorjev s funkcijo cosine_similarity

Vsak playlist predstavlja nekega uporabnika, ki poslusa pesmi na playlistu. S primerjavo vektorjev bomo ugotovili podobnost pesmi 2 uporabnikov. 

Pesmi bomo napovedovali iz podatkov kjer imamo 24 razlicnih playlistov (uporabnikov) razlicnih zanrov.

PLAYLISTS
- 0,1,2,3 - classical
- 4,5,6,7 - rap
- 8,9 - edm
- 10 - techno
- 11 - House
- 12,13,14,15 - rock
- 16,17,18,19 - latin
- 20,21,22,23 - pop

<img src="pictures/Vector/playlist_vector.png" width="700"/>

Vektorju atributov playlista boma dodali se deleze zanrov pesmi, ki jih ta playlist vsebuje. 

Se pravi bomo vektorju dodali 6 vrednosti, vsaka bo predstavljala delez pesmi tega zanra na playlistu. 

Primer: ce imamo playlist klasike bodo vrednosti 0,0,0,0,0,1 (1 predstavlja 100% delez pesmi klasike).

<img src="pictures/Vector/delez_zanrov.png" width="700"/>
<img src="pictures/Vector/podobnost_playlistov_vector.png" width="300"/>


Opazimo, da je primerjava 2 playlistov klasike ali rapa 1, kar pomeni, da metoda primerjanja playlistov deluje. 

Vidimo da je rap playlista nabolj podobna pop playlisti, nato latino, rock, edm in na konca najmanj podobna kalsiki. 

Za edm pa vidimo, da je najbolj podobna popu in rocku. 

Z metodo podobnosti primerjamo se playliste in pesmi. S tem lahko v praksi neakj novo pojavljenih pesmi na platformi, primerjamo s playlisti razlicnih oseb, in pesem predlagamo osebam, ki so najbolj podobni.

Ta metoda (primerjava vektorejv) za playliste in pesmi deluje dobro pri playlistih, kjer imamo pesmi istega zanra (vektor bo izracunan iz povprecja atributov in bo predstavljal to zvrst). Problem pa nastane, kjer imamo na playlistu polovico pesmi 1 zanra in polovico nasprotnega zanra (po vrednotih atributov). V takem primeru, bi se vrednosti atributov pokrajsali in bi dobili neke 3. vrednosti, ki niso podobni ne pesmim 1 zanra ne drugega.

## Uvrscanje pesmi na playlist (KNN)
Cilj: uvrstiti (predlagati) n - novih pesmi, ki se pojavijo na platformi, uporabnikom, ki imajo najvec sosednih pesmi.

1. Nakljucno izberemo n pesmi iz playlistov, te pesmi bomo napovedovali
2. Za vsako pesem izracunamo 100 najbliznjih sosedov vseh playlist
3. Izracunamo delez, na katerih playlistih se pojavi najvec sosed 

Iz 1. playlista nakljucno izberemo 1 pesem, ki jo bomo napovedovali. 

S KNN algoritmom bomo izracunalni njenih 200 najbliznjih sosed (izmed vseh pesmi na vseh playlistih, ki jih imamo).

<img src="pictures/KNN/knn_10pesmi.png" width="700"/>

Za pesem zanra klasika smo poisklai 10 najbolj podobnosih sosedov. Sosedi so iz 0 in 2 playlista, ki sta tudi klasika. 

Pomeni, da napovedi za 10 najbolj podobnih pesmi delujejo vredu.


<img src="pictures/KNN/delez_sosedov1.png" width="400"/>

Pesem bi predlagali playlistu 2,1,0 ker ima pesem na teh playlistih najvecji delez podobnih sosedov.

Za pesem zanra klasika bi uvrstili na playliste 0,1,2 (playlisti klasike), pomeni, da bi pesem napovedali pravemu uporabniku.

## Playlist recommendation system
1. Iskanje podobnih playlistov (KNN mean) - izberemo 3 njabolj podobne
2. Izbira najbolj bliznjih/podobnih pesmi (KNN) - izberemo 2 pesmi iz vsakega playlista
3. Predlagamo 6 najbolj podobnih pesmi (3 PL * 2 pesmi)

Playliste si lahko predstavljamo kot uporabnike, najprej izmed vseh uporabnikov izberemo najbolj podobne, in nato iz vsakega vzamemo po 2 najbolj podbni pesmi in jih predlagamo uporabniku.

S funkcijo 'playlist_similar' poiscemo playliste, ki so najbolj podobni playlist, za katero napovedujemo nove pesmi. 

S KNN metodo poiscemo najbloj podobne sosede nekega playlista in povprecimo njihove razdalje. Tako dibimo povprecno razdaljo (podobnost) do nekega playlista. To moramo preveriti za vse playliste, ki jih imamo.

Za playlist 0 poiscemo najbolj podobne sosede (metoda KNN - najbliznji sosedje, razvrstimo po povprecni razdalji). Na koncu izberemo prve 3.

S funkcijo 'playlist_recommendation' napovedujemo pesmi, ki menimo, da bi jih uporabnik nekega playlista poslusal. 
Napovedane pesmi so podobne pesmim iz playlista uporabnika, in so izvzete iz playlistov drugih uporabnikov (sorodnih).

Naprej poiscemo 3 najbolj podobne (sorodne) playliste, nato izberemo iz vsakega od teh playlistov 2 nabolj primerni pesmi za napoved.

### Klasika

<img src="pictures/KNN/reccomended_songs_klasika.png" width="900"/>

### Rap

<img src="pictures/KNN/reccomended_songs_rap.png" width="700"/>

### Latino

<img src="pictures/KNN/reccomended_songs_latino.png" width="600"/>



UGOTOVITVE: 

Sistem napovedovanja pesmi deluje vredu na izbranih playlistah (24).

Za klasicno playlisto napove klasicne pesmi, za playlisto, kjer je veliko rapa napove rap pesmi...

Playliste si lahko predstavljamo kot uporabnike, najprej izmed vseh uporabnikov izberemo najbolj podobne, in nato iz vsakega vzamemo po 2 najbolj podbni pesmi in jih predlagamo uporabniku.

Sistem zgleda da deluje vredu, uporabniku, ki poslusa rap napove pesmi rapa... Seveda pa je izbira pesmi uporabnika se vedno subjektivna, in na njegovo izbira vpliva vec kot pa le 10 znacilnosti pesmi. 